## Get cleaner annotations and save them as yaml

In [1]:
import yaml

with open('/home/tk/datasets/erc/EmotiW_2018/datasets.yaml', 'r') as stream:
    datasets = yaml.safe_load(stream)

## Instantiate the three classes

In [2]:
from cltl_face_all.face_alignment import FaceDetection
from cltl_face_all.arcface import ArcFace

fd = FaceDetection(device='cpu', face_detector='sfd')
af = ArcFace()

from cltl_face_all.agegender import AgeGender
ag = AgeGender(device='cpu')

[*] load ckpt from /home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/cltl_face_all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt


## Run over the images

In [3]:
import av
import numpy as np
import os

import signal
import time
from tqdm.notebook import tqdm

class Timeout(Exception):
    pass

def handler(sig, frame):
    raise Timeout

signal.signal(signal.SIGALRM, handler) 

for DATASET in tqdm(['train', 'val', 'test']):
    os.makedirs(os.path.join('/home/tk/datasets/erc/EmotiW_2018/visual-features', DATASET), exist_ok=True)

    for diautt, anno in tqdm(datasets[DATASET].items()):
        signal.alarm(10)
        try:
            to_dump = {}
            vidpath = os.path.join('/home/tk/datasets/erc/EmotiW_2018/raw-videos', DATASET, diautt) + '.avi'
            container = av.open(vidpath)
            save_full_path = os.path.join('/home/tk/datasets/erc/EmotiW_2018/visual-features', DATASET, diautt) + '.npy'
        except Exception as e:
            print(e)
            continue

        if os.path.isfile(save_full_path) and os.path.getsize(save_full_path) > 256:
            continue

        for idx, frame in enumerate(container.decode(video=0)):
            signal.alarm(10)
            try:
                to_dump[idx] = []
                numpy_RGB = np.array(frame.to_image())
                batch = numpy_RGB[np.newaxis, ...]
                bboxes = fd.detect_faces(batch)
                # print(f"number of faces detected in the frame {idx} is {len(bboxes[0])}")

                if len(bboxes[0]) == 0:
                    continue

                landmarks = fd.detect_landmarks(batch, bboxes)

                faces = fd.crop_and_align(batch, bboxes, landmarks)
                faces = np.concatenate(faces, axis=0)
                embeddings = af.predict(faces)
                
                ages, genders = ag.predict(faces)

                # print(len(bboxes[0]), len(landmarks[0]), len(ages), len(genders), len(embeddings), len(faces))
                for bb, lm, a, g, emb in zip(bboxes[0], landmarks[0], ages, genders, embeddings):
                    x1, y1, x2, y2, prob = bb

                    to_append = {'bbox': bb, 
                                'landmark': lm, 
                                'age': a,
                                'gender': g,
                                'embedding': emb}

                    to_dump[idx].append(to_append)
                # print(len(to_dump), idx+1)
            except Exception as e:
                print(e)
                continue

        np.save(save_full_path, to_dump)

name 'ONLY_FACE' is not defined
name 'ONLY_FACE' is not defined
name 'ONLY_FACE' is not defined
name 'ONLY_FACE' is not defined
name 'ONLY_FACE' is not defined
name 'ONLY_FACE' is not defined




KeyboardInterrupt: 